In [ ]:
import sqlite3
import pandas as pd
import numpy as np

In [8]:
#Importing DATA
conn = sqlite3.connect('data/raw_data_project_m1.db')

sql_statement='''
          SELECT personal_info.uuid,country_info.country_code, career_info.normalized_job_code, personal_info.age
          FROM personal_info
          JOIN career_info ON personal_info.uuid=career_info.uuid
          JOIN country_info ON personal_info.uuid=country_info.uuid
          JOIN poll_info ON personal_info.uuid=poll_info.uuid
          '''
raw_df=pd.DataFrame(pd.read_sql_query(sql_statement, conn))

raw_df.head()
# raw_df.info()

In [40]:
#Cleaning age

#Exploring values
# age_values=raw_df['age'].unique()
# print(list(age_values))

# cleaning
clean_age=[]
for item in raw_df['age']:
    if "years old" in item:
        clean_age.append(int(item[0:2]))
    elif len(item)>=4:
        clean_age.append(2016-int(item))

raw_df['age']=clean_age #add to dataframe

raw_df['age']=raw_df['age']+4 #convert to 2020

TypeError: argument of type 'int' is not iterable

In [75]:
#Cleaning country_code

#Exploring values
# list(raw_df['country_code'].unique())
#Importing csv of country-country code
country_equiv=pd.read_csv('data/country_code.csv',names=['Country','Code'])
#Converting to dictionary
country_dict={}
for i in country_equiv.index:
    k=country_equiv.loc[i,'Code']
    v=country_equiv.loc[i,'Country']
    country_dict[k[1:3]]=v
#Adding missing values
country_dict['GB']='Great Britain'
country_dict['GR']='Greece'
#Adding country to dataframe
for i in raw_df.index:
    code=raw_df.loc[i,'country_code']
    raw_df.loc[i,'country']=country_dict[code]

In [17]:
#REPORTING

In [ ]:
while True:
    try:
        report_type=int(input('''Please enter which type of report you like:
        (1) To get the table for every country included in the dataset 
        (2) To get the table for a specific country imputed by the user'''))
        break
    except:
        pass